# Evaluation of experiments - E1
fistly define the UUID of the experiment to use

In [1]:
experimentId="37aac705-1a78-4277-9685-3e72d2ed14fe"

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load mass

In [3]:
samplerate=200  #Should rethink this

In [4]:
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate)

In [5]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)
len(annotations)

33458

In [6]:
minDuration=0.3
maxDuration=5
annotations=annotations[annotations.duration>minDuration].reset_index(drop=True)
annotations=annotations[annotations.duration<maxDuration].reset_index(drop=True)
len(annotations)

33269

In [7]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## Load experiment results

In [8]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [9]:
experimentModels.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,aa11eb61-61e6-41e2-94d5-a4833e9b8d9d,0.040017
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,56daf263-9a04-44a8-a967-ed222c84ae8c,0.042790
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,983f3902-7ddc-4ab2-93e7-4da2577f8b4b,0.041762
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,c0eba1b0-cd60-4e9e-ab7f-71131a829c7e,0.037779
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,d7cf3254-9d32-4e20-a23e-3cdcd3107eaa,0.040051


In [10]:
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## Optimal hyperparameter estimation with E1 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [11]:
experimentModels=experimentModels[experimentModels.criteriumName=='E1'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,aa11eb61-61e6-41e2-94d5-a4833e9b8d9d,0.040017
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,56daf263-9a04-44a8-a967-ed222c84ae8c,0.042790
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,983f3902-7ddc-4ab2-93e7-4da2577f8b4b,0.041762
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,c0eba1b0-cd60-4e9e-ab7f-71131a829c7e,0.037779
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,d7cf3254-9d32-4e20-a23e-3cdcd3107eaa,0.040051
5,0001,E1,[0001],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,4f832318-c477-4835-acd3-7ec40944edd3,0.040469
6,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,15e578bc-6917-4cef-a16a-bade7c0b0676,0.037982
7,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,7e1fcff0-30f4-4f29-90d4-f7171b7700cf,0.037858
8,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,0f783e82-ff7a-4204-9e47-55e534671311,0.032807
9,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,99bfe384-9dde-485c-8233-eb9370c0bdbf,0.034677


In [12]:
auxThres=pd.DataFrame({
    'hyperThres':[0.1,0.2,0.3,0.4,0.5,0.6]
})
auxDepth=pd.DataFrame({
    'hyperDepth':[10,20,30,40,50,60]
})
auxMinDuration=pd.DataFrame({
    'hyperMinDuration':[0.3]
})
auxMaxDuration=pd.DataFrame({
    'hyperMaxDuration':[5]
})
auxClose=pd.DataFrame({
    'hyperClose':[0.1]
})

hyperParams=pd.merge(auxThres,auxDepth,how='cross')
hyperParams=pd.merge(hyperParams,auxMinDuration,how='cross')
hyperParams=pd.merge(hyperParams,auxMaxDuration,how='cross')
hyperParams=pd.merge(hyperParams,auxClose,how='cross')

hyperParams

,hyperThres,hyperDepth,hyperMinDuration,hyperMaxDuration,hyperClose
0,0.1,10,0.3,5,0.1
1,0.1,20,0.3,5,0.1
2,0.1,30,0.3,5,0.1
3,0.1,40,0.3,5,0.1
4,0.1,50,0.3,5,0.1
5,0.1,60,0.3,5,0.1
6,0.2,10,0.3,5,0.1
7,0.2,20,0.3,5,0.1
8,0.2,30,0.3,5,0.1
9,0.2,40,0.3,5,0.1


In [13]:
## load hypnograms
stagesAnnotations=pd.read_csv(datapath+"\MASS\stages\stages.csv")
stagesAnnotations['samplerate']=samplerate
stagesAnnotations['subjectId']=stagesAnnotations.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
stagesAnnotations['stopTime']=stagesAnnotations.apply(
    lambda row: row.startTime+row.duration , axis=1)
stagesAnnotations['startInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
stagesAnnotations['stopInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)
stagesAnnotations.head(5)

hypnograms={}
for ind, row in signalsMetadata.iterrows():
    subjectId=row.subjectId
    thisStages=stagesAnnotations[stagesAnnotations.subjectId==subjectId]
    excerptDimension=int(row.duration*row.samplerate)
    thisHypnogram=np.ones((excerptDimension,))*np.nan
    for ind_stg, row_stg in thisStages.iterrows():
        thisHypnogram[row_stg.startInd:row_stg.stopInd]=row_stg.value
    hypnograms[subjectId]=thisHypnogram

In [14]:
hyperExperimentModels=[]
hyperMeanF1=[]
hyperStdF1=[]

for ind_hyper,row_hyper in hyperParams.iterrows():
    print(ind_hyper)
    thisExperimentModels=experimentModels.copy()
    hyperThres=row_hyper.hyperThres
    hyperDepth=int(row_hyper.hyperDepth)
    hyperMinDuration=row_hyper.hyperMinDuration
    hyperMaxDuration=row_hyper.hyperMaxDuration
    hyperClose=row_hyper.hyperClose
    
    meanF1=[]
    meanPrecision=[]
    meanRecall=[]

    stdF1=[]
    stdPrecision=[]
    stdRecall=[]

    for ind,row in thisExperimentModels.iterrows():
        #load model
        model=loadBooster(row.modelId,experimentId,datapath)
        #initialise lists
        rawF1s=[]
        rawPrecisions=[]
        rawRecalls=[]

        f1s=[]
        precisions=[]
        recalls=[]

        #iterate validation subjects
        for valSubjectId in row.val:
            valHypnogram=hypnograms[valSubjectId]
            valN2Mask=valHypnogram==2
            #Define annotations criterium
            usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
            #Load features and labels
            valFeatures=loadFeatureMatrix([valSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
            valLabels=loadLabelsVector([valSubjectId],usedAnnotations,signalsMetadata,samplerate)
            #Predict
            valDMatrix=xgb.DMatrix(data=valFeatures)
            probabilities=model.predict(valDMatrix,iteration_range=(0,hyperDepth))
            raw=probabilities>=hyperThres
            #Apply mask ---------------------------------->
            raw=raw*valN2Mask
            #<--------------------------------------------
            #Processed labels
            processed=labelingProcess(raw,hyperClose,hyperMinDuration,samplerate)
            gtAnnotations=labelVectorToAnnotations(valLabels,samplerate)
            detections=labelVectorToAnnotations(processed,samplerate)
            detections=detections[detections.duration<hyperMaxDuration]
            #Metrics
            f,r,p=annotationPairToMetrics(gtAnnotations,detections)
            
            #Metric appends
            f1s.append(f)
            precisions.append(p)
            recalls.append(r)

        #statistics of the metrics over the subjects of the validation set
        meanF1.append(np.mean(f1s))
        meanPrecision.append(np.mean(precisions))
        meanRecall.append(np.mean(recalls))

        stdF1.append(np.std(f1s))
        stdPrecision.append(np.std(precisions))
        stdRecall.append(np.std(recalls))
        
    thisExperimentModels['meanF1']=meanF1
    thisExperimentModels['meanPrecision']=meanPrecision
    thisExperimentModels['meanRecall']=meanRecall
    
    thisExperimentModels['stdF1']=stdF1
    thisExperimentModels['stdPrecision']=stdPrecision
    thisExperimentModels['stdRecall']=stdRecall

    hyperExperimentModels.append(thisExperimentModels)
    hyperMeanF1.append(np.mean(thisExperimentModels['meanF1']))
    hyperStdF1.append(np.std(thisExperimentModels['meanF1']))
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


In [15]:
hyperParams['meanMeanF1']=hyperMeanF1
hyperParams['stdMeanF1']=hyperStdF1

In [16]:
optimalInd=hyperParams[hyperParams.meanMeanF1==np.max(hyperParams.meanMeanF1)].index[0]
print("maximal mean score at:")
optimal=hyperParams.iloc[optimalInd]
optimal

maximal mean score at:


hyperThres           0.300000
hyperDepth          30.000000
hyperMinDuration     0.300000
hyperMaxDuration     5.000000
hyperClose           0.100000
meanMeanF1           0.742168
stdMeanF1            0.039233
Name: 14, dtype: float64

## Graphical representation of the optimal point

In [17]:
aux=hyperParams[(hyperParams.hyperDepth==optimal.hyperDepth)&(hyperParams.hyperMinDuration==optimal.hyperMinDuration)&
                (hyperParams.hyperClose==optimal.hyperClose)&(hyperParams.hyperMaxDuration==optimal.hyperMaxDuration)].reset_index(drop=True)
px.scatter(aux,x='hyperThres',y='meanMeanF1',error_y='stdMeanF1')

In [18]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperMinDuration==optimal.hyperMinDuration)&
                (hyperParams.hyperClose==optimal.hyperClose)&(hyperParams.hyperMaxDuration==optimal.hyperMaxDuration)].reset_index(drop=True)
px.scatter(aux,x='hyperDepth',y='meanMeanF1',error_y='stdMeanF1')